In [1]:
import pandas as pd
import numpy as np

# from sklearn.datasets import fetch_california_housing

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, RegressionPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, RegressionTestPreset
from evidently.tests import *

In [41]:
data = fetch_california_housing(as_frame=True)
housing_data = data.frame
housing_data.dtypes

MedInc         float64
HouseAge       float64
AveRooms       float64
AveBedrms      float64
Population     float64
AveOccup       float64
Latitude       float64
Longitude      float64
MedHouseVal    float64
dtype: object

In [55]:
housing_data.head(1)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.02381,322.0,2.555556,37.88,-122.23,4.526


In [89]:
# Charger les données
data_train = pd.read_csv("app_train.csv")
data_test = pd.read_csv("app_test.csv")

In [90]:
data_train.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE',
       'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=123)

In [57]:
# Sélectionner uniquement les colonnes numériques
numeric_columns = data_train.select_dtypes(include=['number'])

# Créer un nouveau DataFrame avec les colonnes numériques
df_numeric = data_train[numeric_columns.columns]
df_numeric.dtypes

Unnamed: 0                      int64
SK_ID_CURR                      int64
TARGET                          int64
CNT_CHILDREN                    int64
AMT_INCOME_TOTAL              float64
                               ...   
AMT_REQ_CREDIT_BUREAU_DAY     float64
AMT_REQ_CREDIT_BUREAU_WEEK    float64
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
Length: 107, dtype: object

In [79]:
# Sélectionner les colonnes spécifiques
# selected_columns = ['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT']
selected_columns = ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT']
# Créer un DataFrame avec les colonnes spécifiques
df_selected = data_train[selected_columns]

# Supprimer les lignes contenant des valeurs nulles
df_cleaned = df_selected.dropna()
df_cleaned.dtypes
# df_cleaned contiendra les données nettoyées sans les lignes contenant des valeurs nulles


SK_ID_CURR            int64
TARGET                int64
CNT_CHILDREN          int64
AMT_INCOME_TOTAL    float64
AMT_CREDIT          float64
dtype: object

In [87]:
# Calculer l'âge à partir de la colonne "DAYS_BIRTH" dans le DataFrame client
reference['Age'] = -client_data['DAYS_BIRTH'] // 365

# Vérifiez si la colonne "Age" a été correctement ajoutée
print(reference.head())

KeyError: 'DAYS_BIRTH'

In [62]:
reference.dtypes

Unnamed: 0                      int64
SK_ID_CURR                      int64
target                          int64
CNT_CHILDREN                    int64
AMT_INCOME_TOTAL              float64
                               ...   
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
prediction                    float64
Age                             int64
Length: 109, dtype: object

In [63]:
# Calculer l'âge à partir de la colonne "DAYS_BIRTH" dans le DataFrame client
current['Age'] = -client_data['DAYS_BIRTH'] // 365

# Vérifiez si la colonne "Age" a été correctement ajoutée
print(current.head())

      Unnamed: 0  SK_ID_CURR  target  CNT_CHILDREN  AMT_INCOME_TOTAL  \
5842        5842      106837       0             0          315000.0   
5857        5857      106857       0             1          112500.0   
4416        4416      105161       0             0          176400.0   
8911        8911      110367       0             0          225000.0   
3385        3385      103953       0             0          171000.0   

      AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  \
5842   1971072.0      68512.5        1800000.0                    0.072508   
5857    270000.0      13500.0         270000.0                    0.009549   
4416    921861.0      27085.5         769500.0                    0.022800   
8911    254700.0      27153.0         225000.0                    0.009630   
3385    781695.0      25344.0         652500.0                    0.010500   

      DAYS_BIRTH  ...  FLAG_DOCUMENT_20  FLAG_DOCUMENT_21  \
5842      -18333  ...                

In [64]:
reference.head(10)

,Unnamed: 0,SK_ID_CURR,target,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,prediction,Age
8326,8326,109704,0,2,90000.0,270000.0,13500.0,270000.0,0.007120,-15404,...,0,0,0.0,0.0,0.0,0.0,2.0,3.0,-4.017022,42
7941,7941,109259,1,1,162000.0,1024740.0,52452.0,900000.0,0.019689,-15945,...,0,0,0.0,0.0,0.0,0.0,2.0,4.0,2.463747,43
2757,2757,103216,0,0,135000.0,746280.0,54306.0,675000.0,0.032561,-11630,...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,-6.921971,31
5731,5731,106699,0,0,67500.0,237024.0,12231.0,180000.0,0.018850,-8072,...,0,0,0.0,0.0,0.0,1.0,0.0,1.0,1.144594,22
797,797,100912,0,2,225000.0,254700.0,16276.5,225000.0,0.046220,-10582,...,0,0,0.0,0.0,0.0,0.0,1.0,2.0,6.649800,28
2360,2360,102761,0,0,157500.0,1555569.0,79020.0,1485000.0,0.020246,-19770,...,0,0,2.0,0.0,1.0,0.0,0.0,1.0,-5.723255,54
3795,3795,104436,0,1,121500.0,247500.0,12375.0,247500.0,0.028663,-17837,...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,4.199625,48
2794,2794,103262,0,0,171000.0,948582.0,30600.0,679500.0,0.007274,-15977,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,-1.772702,43
7295,7295,108497,0,0,225000.0,630000.0,31500.0,630000.0,0.014520,-17677,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4.177698,48
6423,6423,107492,0,0,126000.0,380533.5,16893.0,328500.0,0.031329,-21152,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,-3.978430,57


In [141]:
# import warnings
# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')
# # Load Data
# data = fetch_california_housing(as_frame=True)
# housing_data = data.frame
# housing_data.rename(columns={'MedHouseVal': 'target'}, inplace=True)
# housing_data['prediction'] = housing_data['target'].values + np.random.normal(0, 5, housing_data.shape[0])
# reference = housing_data.sample(n=5000, replace=False)
# current = housing_data.sample(n=5000, replace=False)

# Charger les données
data_train = pd.read_csv("app_train.csv")
data_test = pd.read_csv("app_test.csv")

# Assurez-vous d'avoir correctement chargé votre DataFrame client auparavant
# Votre DataFrame client est supposé être stocké dans une variable appelée 'client_data'

# Supprimez ces lignes si elles ne sont pas nécessaires
# Elles désactivent simplement les avertissements de pandas
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Utilisez votre DataFrame client (client_data) à la place de la charge de données California Housing
client_data = data_train.copy()  # Assurez-vous d'avoir une copie de vos données

# Calculer l'âge à partir de la colonne "DAYS_BIRTH" dans le DataFrame client
client_data['Age'] = -client_data['DAYS_BIRTH'] // 365
# Calculer l'âge à partir de la colonne "DAYS_BIRTH" dans le DataFrame client
client_data['Age'] = -client_data['DAYS_BIRTH'] // 365

# Sélectionner les colonnes spécifiques
# selected_columns = ['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT']
selected_columns = ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT','Age']
# Créer un DataFrame avec les colonnes spécifiques
df_selected = client_data[selected_columns]

# Sélectionner uniquement les colonnes numériques
numeric_columns = df_cleaned.select_dtypes(include=['number'])

# Créer un nouveau DataFrame avec les colonnes numériques
client_data = df_cleaned[numeric_columns.columns]

# Assurez-vous que les colonnes correspondent à celles de votre DataFrame client
client_data.rename(columns={'TARGET': 'target'}, inplace=True)

# Remplacez les valeurs NaN par des zéros dans la colonne 'target' du dataframe de référence
client_data['target'].fillna(0, inplace=True)

# Ajoutez une colonne de prédiction basée sur la colonne 'target' avec un bruit aléatoire
client_data['prediction'] = client_data['target'] + np.random.normal(0, 5, client_data.shape[0])

client_data['prediction'].fillna(0, inplace=True)

# Créez un DataFrame de référence et actuel à partir de votre DataFrame client
reference = client_data.sample(n=5000, replace=False)
current = client_data.sample(n=5000, replace=False)

# Affichez les DataFrames de référence et actuels
print("DataFrame de Référence (Reference):")
print(reference)
print("\nDataFrame Actuel (Current):")
print(current)
reference = reference.astype(float)
current = current.astype(float)
# Supprimer les lignes contenant des valeurs nulles
reference = reference.dropna()
current = current.dropna()


DataFrame de Référence (Reference):
      SK_ID_CURR  target  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  Age  \
6610      107713       0             0          180000.0    270000.0   61   
128       100149       0             0          171000.0    539590.5   25   
5213      106099       0             1           76500.0    675000.0   60   
9328      110845       0             0           90000.0    729792.0   43   
113       100133       0             0           67500.0    384048.0   25   
...          ...     ...           ...               ...         ...  ...   
2038      102388       0             0          135000.0    787131.0   43   
8144      109497       0             0           99000.0    239850.0   62   
6109      107142       0             0           90000.0    180000.0   23   
5323      106226       0             0          112500.0    314055.0   52   
5389      106307       0             2          202500.0    358344.0   44   

      prediction  
6610    1.090218  
1

In [142]:
reference.dtypes

SK_ID_CURR          float64
target              float64
CNT_CHILDREN        float64
AMT_INCOME_TOTAL    float64
AMT_CREDIT          float64
Age                 float64
prediction          float64
dtype: object

In [143]:
# Importation des bibliothèques nécessaires
# from datadrift import Report, DataDriftPreset, ColumnSummaryMetric, ColumnQuantileMetric, ColumnDriftMetric, generate_column_metrics

# Définition du rapport initial avec des métriques de référence
report = Report(metrics=[
    DataDriftPreset(),  # Utilise la métrique de dérive de données prédéfinie
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec des métriques spécifiques sur la colonne 'AveRooms'
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),  # Résume les statistiques de la colonne
    ColumnQuantileMetric(column_name='AMT_INCOME_TOTAL', quantile=0.25),  # Calcule le quantile de la colonne
    ColumnDriftMetric(column_name='AMT_INCOME_TOTAL'),  # Calcule la dérive de la colonne
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec des métriques générées pour plusieurs colonnes
report = Report(metrics=[
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile': 0.25}, columns=['AMT_INCOME_TOTAL', 'AMT_CREDIT']),
    # Génère des métriques de quantile pour les colonnes spécifiées
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec une variété de métriques pour différentes colonnes et une métrique prédéfinie
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),  # Résume les statistiques de la colonne
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile': 0.25}, columns='num'),  # Génère des métriques de quantile pour 'num'
    DataDriftPreset(),  # Utilise la métrique de dérive de données prédéfinie
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Conversion du rapport en dictionnaire
report.as_dict()

# Conversion du rapport en format JSON
report.json()

# Explication du code :
# Ce code Python utilise la bibliothèque "datadrift" pour créer des rapports de dérive de données entre un jeu de données de référence (reference) et un jeu de données actuel (current).
# Le code commence par importer les bibliothèques nécessaires.
# Il définit ensuite plusieurs rapports avec différentes métriques pour évaluer la dérive de données, telles que la dérive globale des données, les statistiques de colonnes, les quantiles, etc.
# Chaque rapport est exécuté avec les données de référence et les données actuelles, puis le rapport est affiché.
# À la fin, le code montre comment convertir le rapport en dictionnaire et en format JSON.
# Ces rapports sont utiles pour surveiller et analyser la stabilité des données au fil du temps, ce qui est essentiel dans de nombreuses applications, notamment pour détecter des anomalies ou des changements dans les données.

# # Supposons que reference et current contiennent vos données de référence et actuelles

# # Analyse de la dérive de données
# data_drift_report = e.Report(
#     metrics=[e.DataDriftPreset()]
# )
# data_drift_report.run(reference_data=reference, current_data=current)

# # Analyse de la colonne 'AMT_INCOME_TOTAL'
# income_total_report = e.Report(
#     metrics=[
#         e.ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),
#         e.ColumnQuantileMetric(column_name='AMT_INCOME_TOTAL', quantile=0.25),
#         e.ColumnDriftMetric(column_name='AMT_INCOME_TOTAL')
#     ]
# )
# income_total_report.run(reference_data=reference, current_data=current)

# # Analyse des colonnes 'AMT_INCOME_TOTAL' et 'AMT_CREDIT'
# income_credit_report = e.Report(
#     metrics=[
#         e.generate_column_metrics(e.ColumnQuantileMetric, parameters={'quantile': 0.25}, columns=['AMT_INCOME_TOTAL', 'AMT_CREDIT'])
#     ]
# )
# income_credit_report.run(reference_data=reference, current_data=current)

# # Analyse de la dérive de données avec la colonne 'AMT_INCOME_TOTAL'
# income_data_drift_report = e.Report(
#     metrics=[
#         e.ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),
#         e.generate_column_metrics(e.ColumnQuantileMetric, parameters={'quantile': 0.25}, columns='AMT_INCOME_TOTAL'),
#         e.DataDriftPreset()
#     ]
# )
# income_data_drift_report.run(reference_data=reference, current_data=current)

# Affichage des rapports
# income_total_report.show()
# income_credit_report.show()
# income_data_drift_report.show()


'{"version": "0.4.8", "metrics": [{"metric": "ColumnSummaryMetric", "result": {"column_name": "AMT_INCOME_TOTAL", "column_type": "num", "reference_characteristics": {"number_of_rows": 5000, "count": 5000, "missing": 0, "missing_percentage": 0.0, "mean": 166617.31, "std": 90419.38, "min": 27000.0, "p25": 112500.0, "p50": 144000.0, "p75": 202500.0, "max": 1080000.0, "unique": 207, "unique_percentage": 4.14, "infinite_count": 0, "infinite_percentage": 0.0, "most_common": 135000.0, "most_common_percentage": 11.5}, "current_characteristics": {"number_of_rows": 5000, "count": 5000, "missing": 0, "missing_percentage": 0.0, "mean": 167767.68, "std": 94530.71, "min": 27000.0, "p25": 112500.0, "p50": 148500.0, "p75": 202500.0, "max": 1935000.0, "unique": 216, "unique_percentage": 4.32, "infinite_count": 0, "infinite_percentage": 0.0, "most_common": 135000.0, "most_common_percentage": 10.74}}}, {"metric": "ColumnQuantileMetric", "result": {"column_name": "SK_ID_CURR", "column_type": "num", "quant

In [145]:
current.columns

Index(['SK_ID_CURR', 'target', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'Age', 'prediction'],
      dtype='object')

In [149]:
# Test Suite
tests = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
])

tests.run(reference_data=reference, current_data=current)
tests
suite = TestSuite(tests=[
    NoTargetPerformanceTestPreset(),
])

suite.run(reference_data=reference, current_data=current)
suite

In [150]:
# reference['NAME_CONTRACT_TYPE'] = reference['NAME_CONTRACT_TYPE'].astype(str)
# Créez un ensemble de tests en utilisant les objets de colonne du DataFrame
suite = TestSuite(tests=[
#     TestColumnDrift(df_reference['NAME_CONTRACT_TYPE']),
    TestColumnDrift(column_name='target'),
    TestMeanInNSigmas('Age'),
    NoTargetPerformanceTestPreset(columns=['AMT_INCOME_TOTAL','AMT_CREDIT', 'CNT_CHILDREN'])
])


In [151]:
# Exécutez les tests
suite.run(reference_data=reference, current_data=current)
# 
suite

In [155]:

suite = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
    TestColumnDrift('AMT_CREDIT'),
    TestShareOfOutRangeValues('AMT_CREDIT'),
    DataStabilityTestPreset(),
    RegressionTestPreset()    
])

In [156]:
suite.run(reference_data=reference, current_data=current)
suite


In [157]:
suite.as_dict()
suite.json()

'{"version": "0.4.8", "tests": [{"name": "The Number of Columns With Missing Values", "description": "The number of columns with missing values is 0. The test threshold is lte=0.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": 0}, "value": 0.0}}, {"name": "The Number Of Rows With Missing Values", "description": "The number of rows with missing values is 0. The test threshold is lte=0 \\u00b1 1e-12.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": {"value": 0.0, "relative": 0.1, "absolute": 1e-12}}, "value": 0.0}}, {"name": "Number of Constant Columns", "description": "The number of constant columns is 0. The test threshold is lte=0.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": 0}, "value": 0.0}}, {"name": "Number of Duplicate Rows", "description": "The number of duplicate rows is 0. The test threshold is eq=0 \\u00b1 1e-12.", "status": "SUCCESS", "group": "data_integrity"

In [158]:
suite.save_html('test_suite_t.html')

In [111]:
# Transformer toutes les colonnes en float
reference = reference.astype(float)

In [112]:
reference.dtypes

SK_ID_CURR          float64
target              float64
CNT_CHILDREN        float64
AMT_INCOME_TOTAL    float64
AMT_CREDIT          float64
Age                 float64
prediction          float64
dtype: object

In [138]:
# ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT','Age']
# Importer la bibliothèque Pandas si ce n'est pas déjà fait
import pandas as pd

# Remplacer 'df' par le nom de votre DataFrame si ce n'est pas déjà fait
valeurs_uniques_r = reference['SK_ID_CURR'].unique()

# Afficher les valeurs uniques de la colonne 'target'
print(valeurs_uniques_r)
# Remplacer 'df' par le nom de votre DataFrame si ce n'est pas déjà fait
valeurs_uniques_c = current['SK_ID_CURR'].unique()

# Afficher les valeurs uniques de la colonne 'target'
print(valeurs_uniques_c)

[103521. 106666. 111628. ... 104630. 104221. 102305.]
[100304. 107046. 104517. ... 102224. 102952. 105431.]


In [148]:
import pandas as pd
import numpy as np

# Votre DataFrame
data = reference.copy()

# Vérification de la présence de NaN dans le DataFrame
has_nan = data.isnull().any().any()

# Vérification de la présence de valeurs non numériques dans le DataFrame
is_numeric = data.applymap(np.isreal).all().all()

if not has_nan and is_numeric:
    print("Toutes les colonnes contiennent des valeurs numériques et il n'y a pas de NaN.", has_nan,is_numeric)
else:
    print("Le DataFrame contient des valeurs non numériques ou des NaN.")


Toutes les colonnes contiennent des valeurs numériques et il n'y a pas de NaN. False True
